In [258]:
import json
import requests

import pandas as pd

In [259]:
def find_vacines():
    """ Look HEB open vaccine slots """
    response = requests.get('https://heb-ecom-covid-vaccine.hebdigital-prd.com/vaccine_locations.json')
    response.raise_for_status()

    return pd.DataFrame(response.json()['locations'])

def find_manufacturer(df):
    """ Strip out the vaccine manufacturer """
    def get_manufacturer(l):
        if len(l) > 0:
            return l[0]['manufacturer']
        
        return ''

    df['manufacturer'] = df['slotDetails'].apply(lambda x: get_manufacturer(x))

    return df

def lookup_stores(df, addr_zip, radius):
    """ Fine HEB stores near me """
    query = """
    query StoreSearch($address: String!, $radius: Int!, $fulfillmentChannels: [FulfillmentChannelName]) {
      searchStoresByAddress(
        address: $address
        radiusMiles: $radius
        fulfillmentChannels: $fulfillmentChannels
      ) {
        stores {
          distanceMiles
          store {
            storeNumber
          }
        }
      }
    }
    """

    headers = {
        'Content-Type': 'application/json'
    }

    data = {"query": query}

    variables = {
      "address": addr_zip,
      "radius": radius,
      "fulfillmentChannels": []
    }

    if variables:
        data["variables"] = variables

    url = "https://api-edge.heb-ecom-api.hebdigital-prd.com/graphql"

    response = requests.post(url, headers=headers, data=json.dumps(data))
    response.raise_for_status()

    results = json.loads(response.text)
    df_stores = pd.json_normalize(results['data']['searchStoresByAddress']['stores']).rename(columns={"store.storeNumber": "storeNumber"}).set_index('storeNumber')
    
    return df.set_index('storeNumber').join(df_stores).reset_index()

## Note: Change addr_zip to relevant zip code

In [260]:
addr_zip = "78681"
radius = 100  # miles

df = (
    find_vacines()
    .pipe(find_manufacturer)
    .drop(columns = ['slotDetails'])
    .pipe(lookup_stores, addr_zip, radius)
)

## All open HEB slots

In [261]:
df[df['openTimeslots']>0]

,storeNumber,zip,url,type,street,state,openTimeslots,openAppointmentSlots,name,longitude,latitude,city,manufacturer,distanceMiles
290,NaN,78336,https://heb.secure.force.com/FlexibleScheduler...,offsite,700 W Wheeler Ave,TX,1,1,COVID Vaccination Clinic at Aransas Pass Civic...,NaN,NaN,Aransas Pass,Pfizer,NaN


## Map all open HEB slots

In [262]:
import folium


m = folium.Map(location=[30.2672, -97.7431], tiles="OpenStreetMap", zoom_start=8)

df_available = df[df['openTimeslots']>0].dropna(subset=['longitude', 'latitude'])

for _, r in df_available.iterrows():
    folium.Marker(
        location=[float(r['latitude']), float(r['longitude'])],
        popup=folium.Popup('<a href="' + r['url'] + '" target="_blank">' + r['name'] + '</a>')
    ).add_to(m)
        
m

## Open HEB slots near me

In [263]:
df[(df['openTimeslots']>0) & (df['distanceMiles'].notna())].sort_values(by = ['distanceMiles'])

,storeNumber,zip,url,type,street,state,openTimeslots,openAppointmentSlots,name,longitude,latitude,city,manufacturer,distanceMiles


## HEB stores near me

In [264]:
df[df['distanceMiles'].notna()].sort_values(by = ['distanceMiles'])

,storeNumber,zip,url,type,street,state,openTimeslots,openAppointmentSlots,name,longitude,latitude,city,manufacturer,distanceMiles
93,373.0,78681-3922,None,store,16900 N RANCH ROAD 620,TX,0,0,620 and O'Connor H-E-B,-97.72218,30.50028,ROUND ROCK,,2.363398
228,673.0,78665-1044,None,store,250 UNIVERSITY BLVD.,TX,0,0,University Blvd H-E-B,-97.68859,30.56107,ROUND ROCK,,2.870513
179,580.0,78613-7273,None,store,2800 EAST WHITESTONE,TX,0,0,Parmer and Whitestone H-E-B,-97.78545,30.53280,CEDAR PARK,,3.547342
188,591.0,78664-4677,None,store,1700 EAST PALM VALLEY BLVD,TX,0,0,Round Rock H-E-B plus!,-97.65978,30.51770,ROUND ROCK,,4.083287
152,495.0,78664-7186,None,store,603 LOUIS HENNA BLVD. BLDG A,TX,0,0,Louis Henna Blvd H-E-B,-97.65938,30.48179,ROUND ROCK,,5.357543
222,659.0,78717-5992,None,store,14028 NORTH US 183,TX,0,0,Lakeline H-E-B plus!,-97.80340,30.47786,AUSTIN,,6.040220
82,265.0,78613-1900,None,store,170 E WHITESTONE BLVD,TX,0,0,Hwy 183 and Whitestone Blvd H-E-B,-97.82911,30.52250,CEDAR PARK,,6.193270
100,388.0,78727-3901,None,store,6001 WEST PARMER LANE,TX,0,0,Parmer and McNeil H-E-B,-97.74278,30.44181,AUSTIN,,6.464633
12,34.0,78664-4642,None,store,3750 GATTIS SCHOOL RD,TX,0,0,Gattis School Rd H-E-B,-97.61475,30.49721,ROUND ROCK,,7.073611
75,237.0,78626-5400,None,store,1100 SOUTH IH35,TX,0,0,Georgetown H-E-B,-97.69028,30.63446,GEORGETOWN,,7.210548
